
![Py4Eng](https://dl.dropboxusercontent.com/u/1578682/py4eng_logo.png)

# Object Oriented Programming
## Yoav Ram

# Introduction to OOP


Everything in Python is an object so everything in Python has a class. You can find out which class an object belongs to by using the default property `__class__` or giving it to the function `type`. Even the return of the `type` function has a class - it's of class `type`!

In [5]:
type(5)

int

In [6]:
a = 'Hi'
print(a.__class__)
print(type(a.__class__))

<class 'str'>
<class 'type'>


The basic object type is `object`:

In [7]:
print(object)
print(type(object))

<class 'object'>
<class 'type'>


We can define new classes using the `class` statement:

In [3]:
class MyClass:
    pass

We defined a new class called `MyClass`.
We can now create a new instance of our new class:

In [4]:
my_class1 = MyClass()
my_class2 = MyClass()
print(id(my_class1))
print(type(my_class1))
print(id(my_class2))
print(type(my_class2))

78370072
<class '__main__.MyClass'>
78370016
<class '__main__.MyClass'>


Note that both instances have the same type but a different `id`, so they are not the same object:

In [5]:
my_class1 is my_class2

False

We can check if an object is an instance of a class with the `isinstance` function:

In [6]:
print(isinstance(my_class1, MyClass))
print(isinstance(my_class1, object))
print(isinstance(my_class1, int))

True
True
False


Note that all classes inherit from `object` by default (we'll talk about inheritance later on) so they have all of `object`'s methods and attributes:

In [7]:
dir(my_class1)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__']

In [10]:
help(my_class1) # uses the __doc__ attribute

Help on MyClass in module __main__ object:

class MyClass(builtins.object)
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



# `Point` class

Let's learn more by writing a class for a point in a two dimensional Euclidian space ($\mathbb{R}^2$).

We start with the class definition (`class`) and the constructor (`__init__`) which initialized the attributes of the class instance.

Note:

* The first argument to methods (member functions) is always `self`, a reference to the instance.
* We *assert* that the `__init__` arguments are real numbers using the `isinstance` function.

In [11]:
class Point():
    """A point (x,y) in the Euclidian plane.
    """
    
    def __init__(self, x, y):        
        assert isinstance(x, (int, float)) and isinstance(y, (int, float))
        self.x = float(x)
        self.y = float(y)

In [14]:
origin = Point(0, 0)
print("origin", origin.x, origin.y)

p = Point(1, 2)
print("point", p.x, p.y)

origin 0.0 0.0
point 1.0 2.0


Notice that when we send a `Point` to the console we get:

In [20]:
p

Which is not useful, so we will define how `Point` is represented in the console using `__repr__`:

In [18]:
class Point():
    """A point (x,y) in the Euclidian plane.
    """
    
    def __init__(self, x, y):        
        assert isinstance(x, (int, float)) and isinstance(y, (int, float))
        self.x = float(x)
        self.y = float(y)
        
    def __repr__(self):
        return "Point({}, {})".format(self.x, self.y)

In [21]:
Point(1,2)

Point(1.0, 2.0)

Next up we define a method to add two points. Addition is by elements - $(x_1, y_1) + (x_2, y_2) = (x_1+x_2, y_1+y_2)$.

We also allow to add an `int` or `float`, in which case we add the point to a another point with both coordinates equal to the argument value.

In [41]:
class Point():
    """A point (x,y) in the Euclidian plane.
    """
    
    def __init__(self, x, y):        
        assert isinstance(x, (int, float)) and isinstance(y, (int, float))
        self.x = float(x)
        self.y = float(y)
        
    def __repr__(self):
        return "Point({}, {})".format(self.x, self.y)   
    
    def add(self, other):
        if isinstance(other, self.__class__):
            return Point(self.x + other.x , self.y + other.y)
        elif isinstance(other, (int, float)):
            return Point(self.x + other , self.y + other)
        else:
            raise TypeError("Can't add type {} to {}".format(type(other), type(self)))

In [42]:
Point(1,1).add(Point(2,2))

Point(3.0, 3.0)

In [43]:
Point(1,1).add(2)

Point(3.0, 3.0)

In [44]:
Point(1,1).add('2')

TypeError: Can't add type <class 'str'> to <class '__main__.Point'>

A nicer way to do it is to **overload** the addition operator `+` by defining the addition method name to a name Python reserves for addition - `__add__` (those are double underscores):

In [45]:
class Point():
    """A point (x,y) in the Euclidian plane.
    """
    
    def __init__(self, x, y):        
        assert isinstance(x, (int, float)) and isinstance(y, (int, float))
        self.x = float(x)
        self.y = float(y)
        
    def __repr__(self):
        return "Point({}, {})".format(self.x, self.y)   
    
    def __add__(self, other):
        if isinstance(other, self.__class__):
            return Point(self.x + other.x , self.y + other.y)
        elif isinstance(other, (int, float)):
            return Point(self.x + other , self.y + other)
        else:
            raise TypeError("Can't add type {} to {}".format(type(other), type(self)))

In [46]:
Point(1,1) + Point(2,2)

Point(3.0, 3.0)

In [47]:
Point(1,1) + 2

Point(3.0, 3.0)

We want to be a able to compare `Point`s:

In [48]:
Point(1,2) == Point(2,1)

False

In [49]:
Point(1,2) == Point(1,2)

False

In [50]:
p = Point(0, 0)
p == p

True

In [51]:
Point(1,2) > Point(2,1)

TypeError: unorderable types: Point() > Point()

So `==` checks by identity and `>` is not defined. Let us overload both these operators:

In [52]:
class Point():
    """A point (x,y) in the Euclidian plane.
    """
    
    def __init__(self, x, y):        
        assert isinstance(x, (int, float)) and isinstance(y, (int, float))
        self.x = float(x)
        self.y = float(y)
        
    def __repr__(self):
        return "Point({}, {})".format(self.x, self.y)   
    
    def __add__(self, other):
        if isinstance(other, self.__class__):
            return Point(self.x + other.x , self.y + other.y)
        elif isinstance(other, (int, float)):
            return Point(self.x + other , self.y + other)
        else:
            raise TypeError("Can't add type {} to {}".format(type(other), type(self)))
            
    def __eq__(self, other):
        return (self.x, self.y) == (other.x, other.y)
    
    def __gt__(self, other):
        return self.x > other.x and self.y > other.y

First we check if two points are equal:

In [53]:
Point(1,0) == Point(1,2)

False

In [54]:
Point(1,0) == Point(1,0)

True

Then if one is *strictly* smaller than the other:

In [55]:
Point(1,0) > Point(1,2)

False

In [56]:
Point(5,6) > Point(1,2)

True

The addition operator `+` returns a **new instance**. 

Next we will write a method that instead of returning a new instance, changes the current instance:

In [57]:
class Point():
    """A point (x,y) in the Euclidian plane.
    """
    
    def __init__(self, x, y):        
        assert isinstance(x, (int, float)) and isinstance(y, (int, float))
        self.x = float(x)
        self.y = float(y)
        
    def __repr__(self):
        return "Point({}, {})".format(self.x, self.y)   
    
    def __add__(self, other):
        if isinstance(other, self.__class__):
            return Point(self.x + other.x , self.y + other.y)
        elif isinstance(other, (int, float)):
            return Point(self.x + other , self.y + other)
        else:
            raise TypeError("Can't add type {} to {}".format(type(other), type(self)))
            
    def __eq__(self, other):
        return (self.x, self.y) == (other.x, other.y)
    
    def __gt__(self, other):
        return self.x > other.x and self.y > other.y
    
    def increment(self, other): 
        """Add other to self, in-place.
        """
        if isinstance(other, self.__class__):
            self.x += other.x
            self.y += other.y
        else:
            raise TypeError("Can't add type {} to {}".format(type(other), type(self)))

In [58]:
p = Point(6.5, 7)
p + Point(1,2)
print(p)
p.increment(Point(1,2))
print(p)

Point(6.5, 7.0)
Point(7.5, 9.0)


We now write a method that given many points, checks if the current point is more extreme than the other points.

Note that the argument `*points` means that more than one argument may be given.

In [69]:
class Point():
    """A point (x,y) in the Euclidian plane.
    """
    
    def __init__(self, x, y):        
        assert isinstance(x, (int, float)) and isinstance(y, (int, float))
        self.x = float(x)
        self.y = float(y)
        
    def __repr__(self):
        return "Point({}, {})".format(self.x, self.y)   
    
    def __add__(self, other):
        if isinstance(other, self.__class__):
            return Point(self.x + other.x , self.y + other.y)
        elif isinstance(other, (int, float)):
            return Point(self.x + other , self.y + other)
        else:
            raise TypeError("Can't add type {} to {}".format(type(other), type(self)))
            
    def __eq__(self, other):
        return (self.x, self.y) == (other.x, other.y)
    
    def __gt__(self, other):
        return self.x > other.x and self.y > other.y
    
    def increment(self, other): 
        """Add other to self, in-place.
        """
        if isinstance(other, self.__class__):
            self.x += other.x
            self.y += other.y
        else:
            raise TypeError("Can't add type {} to {}".format(type(other), type(self)))
    
    def is_extreme(self, *points):
        return all(map(lambda point: self > point, points))

Note that specifying a function argument with a `*` before its name says that we can give zero or more values and they will be packed in a `tuple`.

In [70]:
p = Point(5, 6)
p.is_extreme(Point(1,1))

True

In [71]:
p.is_extreme(Point(1,1), Point(2,5), Point(6,2))

False

We can also use the method via the class instead of the instance, and give the instance of interest (the one that we want to know if it is the extreme) as the first argument `self`. Much like this, we can either do `'hi'.upper()` or `str.upper('hi')`.

In [72]:
Point.is_extreme(Point(7,8), Point(1,1), Point(4,5), Point(2,3))

True

## Exercise

Overload the `__sub__` method to implement substracting one point from the other. No need to support other types, just `Point`.

In [ ]:
Point(1,1) - Point(1,1)

## `Rectangle` class

We will implement two classes for rectangles, and compare the two implementations.

### First implementation - two points

The first implementation defines a rectangle by its lower left and upper right vertices.

In [73]:
class Rectangle1:
    """Describe a parallel-axes rectangle by storing two points.
    
    Attributes
    ----------
    llv : Point
        lower left vertex
    urv : Point
        upper right vertex
    """
    
    def __init__(self, lower_left_vertex, upper_right_vertex):
        assert isinstance(lower_left_vertex, Point)
        assert isinstance(upper_right_vertex, Point)
        assert lower_left_vertex < upper_right_vertex 
        self.llv = lower_left_vertex
        self.urv = upper_right_vertex
        
    def __repr__(self):
        representation = "Rectangle with lower left {0} and upper right {1}"
        return representation.format(self.llv, self.urv)

    def dimensions(self):
        height = self.urv.y - self.llv.y
        width = self.urv.x - self.llv.x
        return height, width
    
    def area(self):
        height, width = self.dimensions()
        area = height * width
        return area
    
    def copy(self):
        return Rectangle1(self.llv, self.urv)
    
    def transpose(self):
        """Reflection with regard to the line passing through lower left vertex with angle 315 (-45) degrees
        """
        t_rect = self.copy()
        height, width = t_rect.dimensions()
        t_rect.urv = t_rect.llv
        t_rect.llv = Point(t_rect.urv.x - height, t_rect.urv.y - width)
        return t_rect

In [74]:
rect = Rectangle1(Point(0, 0), Point(2,1))
print(rect)
print("Area:", rect.area())
print("Dimensions:", rect.dimensions())
t_rect = rect.transpose()
print("Transposed:", t_rect)

Rectangle with lower left Point(0.0, 0.0) and upper right Point(2.0, 1.0)
Area: 2.0
Dimensions: (1.0, 2.0)
Transposed: Rectangle with lower left Point(-1.0, -2.0) and upper right Point(0.0, 0.0)


### Second implementation - point and dimensions

The second implementation defines a rectangle by the lower left point, the height and the width.

We define the exact same methods as in `Rectangle1`, with the same input and output, but  different inner representation / implementation.

In [75]:
class Rectangle2:
    """Describe a parallel-axes rectangle by storing lower left point, height and width.
    
    Attributes
    ----------
    point : Point
        lower left point
    width : float
        width
    height : float
        height
    """
    def __init__(self, point, height, width):
        assert isinstance(point, Point)
        assert isinstance(height, (int, float))
        assert isinstance(width, (int, float))
        assert height > 0
        assert width > 0        
        self.point = point
        self.height = float(height)
        self.width = float(width)
        
    def __repr__(self):
        representation = "Rectangle with lower left {0} and upper right {1}"
        return representation.format(self.point, Point(self.point.x + self.width, self.point.y + self.height))
    
    def dimensions(self):
        return self.height, self.width

    def area(self):
        area = self.height * self.width
        return area

    def copy(self):
        return Rectangle2(self.point, self.height, self.width)
    
    def transpose(self):
        new_rect = self.copy()
        new_rect.point = Point(new_rect.point.x - new_rect.height , new_rect.point.y - new_rect.width)
        new_rect.height, new_rect.width = new_rect.width, new_rect.height
        return new_rect

In [76]:
rect = Rectangle2(Point(0, 0), 1, 2)
print(rect)
print("Area:", rect.area())
print("Dimensions:", rect.dimensions())
t_rect = rect.transpose()
print("Transposed:", t_rect)

Rectangle with lower left Point(0.0, 0.0) and upper right Point(2.0, 1.0)
Area: 2.0
Dimensions: (1.0, 2.0)
Transposed: Rectangle with lower left Point(-1.0, -2.0) and upper right Point(0.0, 0.0)


## Exercise

Overload the `__contains__` method in one of the rectangle classes with a method that, given a point, returns `True` if the point is inside the rectangle and `False` otherwise.

In [84]:
rect = Rectangle2(Point(0, 0), 1, 2) # or use Rectangle1 if you prefer
assert Point(0.5, 0.5) in rect
assert Point(1.5, 0.5) in rect
assert Point(2.5, 0.5) not in rect
assert Point(0.5, 2.5) not in rect

# Inheritance

In [109]:
class Door:
    OPEN = 'open'
    CLOSED = 'closed'
    
    def __init__(self, status=CLOSED):
        self.status = status

    def __repr__(self):
        return 'Door {0} is {1}'.format(id(self), self.status)
    
    def open(self):
        self.status = self.OPEN
        
    def close(self):
        self.status = self.CLOSED

In [110]:
door = Door()
door

Door 81214544 is closed

In [111]:
from getpass import getpass

class SecurityDoor(Door):
    """A door that requires a password to open.
    """
    
    def __init__(self, password):
        super(SecurityDoor, self).__init__()
        self.password = password
        
    def open(self):
        if getpass("What is the password?") == self.password:
            super(SecurityDoor, self).open()
        else:
            print("Wrong password!")

In [112]:
secure_door = SecurityDoor('opensesame')
print(secure_door)
secure_door.close()
print(secure_door)

What is the password?········
Wrong password!


Door 81215048 is closed

# Composition and Delegation

Composition means that an object knows another object, and explicitly delegates some tasks to it. While inheritance is implicit, composition is explicit: in Python, however, things are far more interesting than this =).

First of all let us implement classic composition, which simply makes an object part of the other as an attribute

In [115]:
from getpass import getpass

class SecurityDoor:
    """A door that requires a password to open.
    """
    
    def __init__(self, password):
        self.door = Door(status=Door.CLOSED)
        self.password = password
            
    def __repr__(self):
        return repr(self.door) 
               
    def open(self):
        if getpass("What is the password?") == self.password:
            self.door.open()
        else:
            print("Wrong password!")
        
    def close(self):
        self.door.close

In [116]:
secure_door = SecurityDoor('opensesame')
secure_door.open()
print(secure_door)
secure_door.close()
print(secure_door)

What is the password?········


Door 78372256 is open



The primary goal of composition is to relax the coupling between objects. This little example shows that now SecurityDoor is an object and no more a Door, which means that the internal structure of Door is not copied. For this very simple example both Door and SecurityDoor are not big classes, but in a real system objects can very complex; this means that their allocation consumes a lot of memory and if a system contains thousands or millions of objects that could be an issue.

The composed SecurityDoor has to redefine every attribute since the concept of delegation applies only to methods and not to attributes, doesn't it?

Well, no. Python provides a very high degree of indirection for objects manipulation and attribute access is one of the most useful. Accessing attributes is ruled by a special method called __getattribute__() that is called whenever an attribute of the object is accessed. Overriding __getattribute__(), however, is overkill; it is a very complex method, and, being called on every attribute access, any change makes the whole thing slower.

The method we have to leverage to delegate attribute access is __getattr__(), which is a special method that is called whenever the requested attribute is not found in the object. So basically it is the right place to dispatch all attribute and method access our object cannot handle. The previous example becomes

In [120]:
from getpass import getpass

class SecurityDoor:
    """A door that requires a password to open.
    """
    
    def __init__(self, password):
        self.door = Door(status=Door.CLOSED)
        self.password = password         
               
    def __repr__(self):
        return repr(self.door)
    
    def open(self):
        if getpass("What is the password?") == self.password:
            self.door.open()
        else:
            print("Wrong password!")
        
    def __getattr__(self, attr):
        return getattr(self.door, attr)

In [121]:
secure_door = SecurityDoor('opensesame')
secure_door.open()
print(secure_door)
secure_door.close()
print(secure_door)

What is the password?········
Door 81135936 is open
Door 81135936 is closed


As this last example shows, delegating every member access through __getattr__() is very simple. Pay attention to getattr() which is different from __getattr__(). The former is a built-in that is equivalent to the dotted syntax, i.e. getattr(obj, 'someattr') is the same as obj.someattr, but you have to use it since the name of the attribute is contained in a string.

Composition provides a superior way to manage delegation since it can selectively delegate the access, even mask some attributes or methods, while inheritance cannot. In Python you also avoid the memory problems that might arise when you put many objects inside another; Python handles everything through its reference, i.e. through a pointer to the memory position of the thing, so the size of an attribute is constant and very limited.

## Exercise

Define a new class, `RevolvingDoor`, which closes immediately after it is opened. Define it once with inheritance and once with composition.

In [ ]:
class RevolvingDoor(Door):
    pass

rdoor = RevolvingDoor()
rdoor.open()
print(rdoor)

In [ ]:
class RevolvingDoor:
    pass
    
rdoor = RevolvingDoor()
rdoor.open()
print(rdoor)

# Polymorphism

In Python, polymorphism is baked into the language, due to the **Duck typing** principle. We saw above how this relates to methods like `__add__`, `__sub__`, `__repr__`, and `__contains__`.

Another example is a file object:

In [148]:
f = open(r"..\data\crops.txt")
print(type(f))
print(dir(f))
f.close()

<class '_io.TextIOWrapper'>
['_CHUNK_SIZE', '__class__', '__del__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__iter__', '__le__', '__lt__', '__ne__', '__new__', '__next__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '_checkClosed', '_checkReadable', '_checkSeekable', '_checkWritable', '_finalizing', 'buffer', 'close', 'closed', 'detach', 'encoding', 'errors', 'fileno', 'flush', 'isatty', 'line_buffering', 'mode', 'name', 'newlines', 'read', 'readable', 'readline', 'readlines', 'seek', 'seekable', 'tell', 'truncate', 'writable', 'write', 'writelines']


Say we write a function that gets a file and returns all the lines that start with a given prefix (say, 'Am'):

In [168]:
def filter_by_prefix(file, prefix):
    return [line.strip() for line in file if line.startswith(prefix)]

with open(r'..\data\crops.txt', 'r') as f:
    print(filter_by_prefix(f, 'Am'))

['Amaranthus cruentus', 'Amaranthus spp.', 'Amaranthus viridis', 'Amomum subulatum', 'Amorphophallus konjac', 'Amorphophallus paeoniifolius']


But now, say we want to read from `crops.txt.gz`, which is compressed with gzip. There is a module for reading gzipped files:

In [169]:
import gzip

In [170]:
gzfile = gzip.open(r'..\data\crops.txt.gz' ,'r')
print(type(gzfile))
print(dir(gzfile))
print(isinstance(gzfile, f.__class__))
gzfile.close()

<class 'gzip.GzipFile'>
['__abstractmethods__', '__class__', '__del__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__next__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '_abc_cache', '_abc_negative_cache', '_abc_negative_cache_version', '_abc_registry', '_add_read_data', '_checkClosed', '_checkReadable', '_checkSeekable', '_checkWritable', '_check_closed', '_init_read', '_init_write', '_new_member', '_read', '_read_eof', '_read_exact', '_read_gzip_header', '_unread', '_write_gzip_header', 'close', 'closed', 'detach', 'extrabuf', 'extrasize', 'extrastart', 'filename', 'fileno', 'fileobj', 'flush', 'isatty', 'max_read_chunk', 'min_readsize', 'mode', 'mtime', 'myfileobj', 'name', 'offset', 'peek', 'read', 'read1', 'readable', 'readinto', 'readlin

You notice that this is a different type than a file we opened with `open` (that would be `f`). 

But our function doesn't care about the type, all it wants is an object that you can loop over with a `for`: an iterable object that implements the `__iter__` method by either returning an iterator (which is an object that implements the `next`) or using the `yield` statement.

In [171]:
with gzip.open(r'..\data\crops.txt.gz', 'rt') as fgz: # rt is for reading text
    print(filter_by_prefix(fgz, 'Am'))

['Amaranthus cruentus', 'Amaranthus spp.', 'Amaranthus viridis', 'Amomum subulatum', 'Amorphophallus konjac', 'Amorphophallus paeoniifolius']


It doesn't even have to be a file. A list is just as good:

In [173]:
continents = ['America', 'Europe', 'Asia', 'Africa', 'Anarctica', 'Australia']
print(filter_by_prefix(continents, 'Am'))

['America']


The [itertools](https://docs.python.org/3.4/library/itertools.html) module has some nice functions for creating clever iterators. For example, one can iterate all the permutations of another iterable:

In [176]:
import itertools

for perm in itertools.permutations(['I', 'Love', 'Python']):
    print(perm)

('I', 'Love', 'Python')
('I', 'Python', 'Love')
('Love', 'I', 'Python')
('Love', 'Python', 'I')
('Python', 'I', 'Love')
('Python', 'Love', 'I')


## Exercise

Let's write a new class called `RandomList` that is the same as the regular list, only that if you iterate over it, the iteration is random, in the sense that the order of the elements changes from one iteration to the next (but not in the middle of the iteration). **Hint** use the function `random.shuffle` frim the `random` module to randomly permute a list in place.

In [211]:
class RandomList:
    pass

In [ ]:
lst = RandomList([1, 2, 3])
lst.append(4)
for n in lst:
    print(n)

# References
 
- Python 3 [OOP tutorial](https://docs.python.org/3/tutorial/classes.html)
- Leonardo Giordani's [OOP notebooks](http://nbviewer.jupyter.org/github/lgiordani/blog_source/blob/master/pelican/content/notebooks/Python_3_OOP_Part_3__Delegation__composition_and_inheritance.ipynb)

## Colophon
This notebook was written by [Yoav Ram](http://www.yoavram.com) and is part of the _Python for Engineers_ course.

The notebook was written using [Python](http://pytho.org/) 3.4.4, [IPython](http://ipython.org/) 4.0.3 and [Jupyter](http://jupyter.org) 4.0.6.

This work is licensed under a CC BY-NC-SA 4.0 International License.

![Python logo](https://www.python.org/static/community_logos/python-logo.png)